# 🔍 Assignment: Build a Context-Aware LLM Agent Using LangChain

##  Objective
Design and implement your **own LLM-powered agent** using the LangChain ecosystem. This agent should:
- Use at least **one tool**
- Incorporate **function-calling**
- Integrate **LangSmith** for observability
- Implement a **fallback mechanism**
- Bonus: Use the **Agentic** library to define a clear multi-step workflow




```

```

##  Your Task
- Choose a use case (e.g., travel planner, medical assistant, customer support bot, product recommender, etc.)
- Build a custom tool (e.g., API wrapper, knowledge lookup, calculator)
- Use LangChain agents to manage interaction
- Integrate LangSmith to track runs
- Handle failure gracefully with a fallback approach
- Optionally add structured workflows with Agentic


# ⛳ Your Trusted Travel Planner



### Step 1: Install necessary Libraries!

In [ ]:
#necessary pip installations
!pip install langchain langchain_openai langchain-community python-dotenv gradio agentic
!pip install langsmith

In [103]:
# Imports
# Note : libraries will be installed depending on requirement
import os
import requests
import uuid
import gradio as gr
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langsmith import traceable


##  Step 2: Set Up Keys
 You’ll need:
- An OpenAI API key (for LLM calls)/ Groq API
- A LangSmith API key (for logging and observability)

In [104]:
#from google.colab import userdata
#OPENAI_API_KEY = userdata.get('OpenAI')
#TAVILY_API_KEY = ('Tavily')
#RAPIDAPI_KEY = ('RapidAPI')

In [105]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
RAPIDAPI_KEY = os.getenv("RAPIDAPI_KEY")

#I'm using an .env file here (made in notepad++) to store environment variables

In [106]:
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file or environment variables.")

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0, api_key=OPENAI_API_KEY)

In [107]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "YOUR_LANGSMITH_API_KEY" # Replace with your actual LangSmith API Key
os.environ["LANGCHAIN_PROJECT"] = "MyTravelAgent"

##  Step 3: Define Custom Tools
 Build at least one tool (e.g., API, calculator, mock database lookup). Use decorators or `Tool` wrappers.

### === TOOL 1: Destination Recommender_ Function as a Tool ===

In [108]:
def recommend_destinations(country: str) -> str:
    suggestions = {
        "France": "Paris, Nice, Lyon, Bordeaux",
        "Japan": "Tokyo, Kyoto, Osaka, Hokkaido",
        "Italy": "Rome, Venice, Florence, Milan",
        "USA": "New York, Los Angeles, Miami, Chicago",
        "UAE": "Dubai, Abu Dhabi",
        "Thailand": "Bangkok, Phuket, Chiang Mai",
        "Turkey": "Istanbul, Cappadocia, Antalya",
        "UK": "London, Edinburgh, Bath, Manchester"
    }
    return suggestions.get(country, "Sorry, I don’t have info for that country.")

destination_tool = Tool(
    name="DestinationRecommender",
    func=recommend_destinations,
    description="Suggests travel cities within a given country"
)

### === TOOL 2: Hotel Booking via RapidAPI ===

In [112]:
import requests

def create_hotel_booking(location, check_in, check_out, budget, adults=1, children=0, rooms=1):
    url = "https://booking-com15.p.rapidapi.com/v1/hotels/search"

    querystring = {
        "location": location,
        "checkin_date": check_in,
        "checkout_date": check_out,
        "adults": str(adults),
        "children": str(children),
        "rooms": str(rooms),
        "currency": "USD",
        "price_max": str(budget)
        # You can add more parameters as needed
    }

    headers = {
        'x-rapidapi-key': "f39e410c6bmshad955ee7a034f9dp103e71jsn04ceecae5446",
        'x-rapidapi-host': "booking-com15.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json()  # Process this response as needed
    else:
        return {"error": response.status_code, "message": response.text}

# Example usage:
booking_results = create_hotel_booking(
    "Tokyo, Japan",
    "2025-12-25",
    "2026-01-03",
    2000,
    adults=2,           # Specify number of adults
    children=0,         # Specify number of children
    rooms=1             # Specify number of rooms
)
print(booking_results)

{'error': 404, 'message': '{"message":"Endpoint \'\\/v1\\/hotels\\/search\' does not exist"}'}


In [113]:
hotel_tool = Tool(
    name="HotelBooking",
    func=create_hotel_booking,
    description=(
        "Use this to book hotels. Requires:\n"
        "- city: the city you're traveling to\n"
        "- check_in: check-in date (YYYY-MM-DD)\n"
        "- check_out: check-out date (YYYY-MM-DD)\n"
        "- guests: number of people\n"
        "- budget: low, medium, or high (optional)"
    )
)

### === TOOL 3: Tavily Travel Search ===

In [114]:
!pip install -U langchain-tavily
from langchain_tavily import TavilySearch

In [115]:
tavily_tool = TavilySearchResults(api_key=TAVILY_API_KEY)
tavily_search_tool = Tool(
    name="TavilySearch",
    func=tavily_tool.run,
    description="Use this to get real-time info about attractions, weather, food, culture, and more in a city."
)

##  Step 4: Enable Function-Calling
 Define structured functions for your agent to call. Use `openai.FunctionsAgent` or LangChain's `OPENAI_FUNCTIONS` agent type.

### === DEFINE AGENT ===

In [116]:

all_tools = [destination_tool, hotel_tool, tavily_search_tool]

MyTravelAgent = initialize_agent(
    tools=all_tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

##  Step 5: Fallback Strategy
 Define a fallback agent or tool to be used when the main tool or agent fails.

Step1: Use 2 diff models as per user query context

In [117]:
from langchain_openai import ChatOpenAI

nano_llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0, api_key=OPENAI_API_KEY)
power_llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=OPENAI_API_KEY)


Step2: Mention all tools

In [118]:
all_tools = [destination_tool, hotel_tool, tavily_search_tool]


Step3: Create two agents!

In [119]:
from langchain.agents import initialize_agent, AgentType

nano_agent = initialize_agent(
    tools=all_tools,
    llm=nano_llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

power_agent = initialize_agent(
    tools=all_tools,
    llm=power_llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)


 Step 4: Add fallback logic to route the query

In [120]:
def smart_travel_agent(user_input: str):
    try:
        if len(user_input.split()) <= 20:
            print("💡 Using lightweight model (gpt-4.1-nano)...")
            return nano_agent.run(user_input)
        else:
            print("⚙️ Using powerful GPT-4 model...")
            return power_agent.run(user_input)

    except Exception as e:
        print(f"⚠️ Error occurred: {e}")
        print("🔁 Falling back to GPT-4...")
        return power_agent.run(user_input)


##  Step 6: Integrate LangSmith
 Wrap your LLM or agent with LangSmith to observe performance and execution flows.


### === LangSmith Tracing ===

In [121]:
@traceable(name="MyTravelAgent")
def run_travel_agent(query):
    return MyTravelAgent.run(query)

# === EXAMPLE CALL ===
if __name__ == "__main__":
    user_query = "Plan a trip to Japan in December. Suggest places, what to pack and key info. Give me a detail of dishes according to the famous places in Japan. book a hotel for me in tokyo. I would Like to spend my New Year."
    print(run_travel_agent(user_query))





> Entering new AgentExecutor chain...
Planning a trip to Japan in December is a fantastic idea! Japan offers a rich blend of traditional culture and modern attractions, especially during the winter season. Here's a detailed plan for your trip:

**Places to Visit in Japan (December):**
1. Tokyo – Explore the vibrant city, visit temples, shopping districts, and enjoy New Year celebrations.
2. Kyoto – Experience traditional temples, shrines, and beautiful winter scenery.
3. Osaka – Known for its delicious street food and lively atmosphere.
4. Hakone – Enjoy hot springs and views of Mount Fuji.
5. Nara – Visit the historic parks and temples.

**What to Pack:**
- Warm clothing: thermal wear, sweaters, a winter coat
- Comfortable walking shoes
- Umbrella and raincoat
- Travel adapter for electronics
- Personal toiletries
- Camera for capturing scenic views
- Holiday gifts and souvenirs

**Key Info for December in Japan:**
- Weather: Cold, especially in northern regions and mountainous area

In [122]:
print("Visit your LangSmith project dashboard here: https://smith.langchain.com/")
print("After logging in, you should see your 'MyTravelAgent' project and its traces.")

Visit your LangSmith project dashboard here: https://smith.langchain.com/
After logging in, you should see your 'MyTravelAgent' project and its traces.


# === Step7: GRADIO UI ===

In [123]:

def respond_to_user(query):
    return run_travel_agent(query)

demo = gr.Interface(
    fn=respond_to_user,
    inputs=gr.Textbox(label="Ask your travel assistant anything"),
    outputs=gr.Textbox(label="Response"),
    title="🌍 AI Travel Agent",
    description="Plan trips, get packing tips, book hotels, and explore cities."
)

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62c6aaf37b1d692764.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


##  (Bonus) Step 8: Define a Workflow using Agentic
 Use the `agentic` library to define a step-by-step flow your agent follows.

 Submission Checklist
- [ ] Agent runs end-to-end on your chosen use case
- [ ] At least one custom tool is integrated
- [ ] Function-calling works with tools
- [ ] LangSmith integration logs all calls
- [ ] Fallback logic is demonstrated
- [ ] Bonus: Agentic workflow is defined

##  Grading (100 pts )


##  Tip
You are free to choose **any use case** — be creative but keep your design modular and well-commented!